In [1]:
from collections import namedtuple
import csv
from time import sleep

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource

In [2]:
Landmark = namedtuple('Landmark', 'i x y')

# speed in m/s , turnrate in rad/s
Control = namedtuple('Control', 'speed turnrate')

# Observed landmark position the x and y distance in m relative to the vehicle
# x - right is positive, y - forward is positive
Observation = namedtuple('Observation', 'i x y')

# Global x and y positions in m, 
# theta  is global vehicle yaw (rad)
GroundTruth = namedtuple('GroundTruth', 'i x y theta')

In [3]:
def get_groundtruths_from_csv(filename):
    groundtruths = []
    with open(filename, newline='') as csv_file:
        reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
        x, y, theta, i_ = 0.0, 0.0, 0.0, 0
        for r in reader:
            x_, y_, theta_= float(r[0]), float(r[1]), float(r[2])
            groundtruth = GroundTruth(i=i_, x=x_, y=y_, theta=theta_)
            groundtruths.append(groundtruth)
            i_+=1
        return groundtruths
    

def get_landmarks_from_csv(filename):
    landmarks = []
    with open(filename, newline='') as csv_file:
        reader = csv.reader(csv_file, delimiter='\t', quotechar='|')
        x, y, i = 0.0, 0.0, 0
        for r in reader:
            x_, y_, i_ = float(r[0]), float(r[1]), int(r[2])
            landmark = Landmark(i=i_, x=x_, y=y_)
            landmarks.append(landmark)
        return landmarks


def get_observations_from_csv(filename):
    observations = []
    with open(filename, newline='') as csv_file:
        reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
        x, y, i_ = 0.0, 0.0, 0
        for r in reader:
            x_, y_ = float(r[0]), float(r[1])
            observation = Observation(i=i_, x=x_, y=y_)
            observations.append(observation)
            i_+=1
        return observations

In [4]:
landmarks = get_landmarks_from_csv("../data/map_data.txt")
groundtruths = get_groundtruths_from_csv("../data/gt_data.txt")

total_observations = []
for i in range(1, 2444):
    file_path ='../data/observation/observations_{:06d}.txt'.format(i)
    observations = get_observations_from_csv(file_path)
    total_observations.append(observations)

In [5]:
BACKGROUND_COLOR = "#222f3e"
LEGEND_TEXT_COLOR = "#a29bfe"
ROBOT_LINE_COLOR = "#833471"
ROBOT_FILL_COLOR = "#9980FA"
LANDMARK_COLOR = "#fd79a8"

def stylize_plot(plot):
    #plot.legend.background_fill_color = "navy"
    plot.axis.major_tick_line_color = None
    plot.axis.major_label_standoff = 0
    plot.grid.grid_line_color = None
    plot.background_fill_color = BACKGROUND_COLOR
    plot.outline_line_color = BACKGROUND_COLOR
    plot.border_fill_color = BACKGROUND_COLOR
    plot.legend.label_text_color = LEGEND_TEXT_COLOR
    plot.legend.background_fill_alpha = 0.0
    plot.legend.label_text_alpha = 1.0
    plot.legend.label_text_font = "courier"
    plot.legend.orientation = "vertical"
    plot.legend.location = "bottom_right"
    
def map_figure(x1=-60, x2=310, y1=-110, y2=60, w=900, h=600):
    return figure(x_range=[x1, x2], y_range=[y1, y2],
        plot_height=h, plot_width=w, 
        x_axis_location=None, y_axis_location=None, tools="")

def extract_landmarks(landmarks):
    x = [landmark.x for landmark in landmarks]
    y = [landmark.y for landmark in landmarks]
    return x, y

def extract_observations(observations):
    x = [observation.x for observation in observations]
    y = [observation.y for observation in observations]
    return x, y

def plot_landmarks(plot, landmarks):
    landmark_x, landmark_y = extract_landmarks(landmarks)
    landmark_source = ColumnDataSource({'x': landmark_x, 'y': landmark_y} )
    plot.square('x', 'y', legend="LANDMARKS", source=landmark_source,
        size=9, line_color=LANDMARK_COLOR, fill_alpha=0.0, line_width=1)
    
def plot_initial_robot(plot, robot_source):
    
    robot_triangle = plot.triangle('x', 'y', legend = "GROUND TRUTH LOCATION", source = robot_source,
        size=30, fill_color=ROBOT_FILL_COLOR, line_color=ROBOT_LINE_COLOR, fill_alpha=1.0, line_width=1,
        angle='theta')

    robot_cross = plot.cross('x', 'y', legend="GROUND TRUTH LOCATION", source=robot_source,
        size=12, line_color=ROBOT_LINE_COLOR, angle='theta', fill_alpha=0.0, line_width=1)

    robot_range = plot.circle('x', 'y', legend="OBSERVATION RANGE", source=robot_source,
        radius=60, line_color=ROBOT_LINE_COLOR, line_dash="10 5", fill_alpha=0.0, line_width=1)

    return (robot_range, robot_triangle, robot_cross)

def plot_robot(source, x, y, theta):
    source.data['x'] = [x]
    source.data['y'] = [y]
    source.data['theta'] = [theta] 

In [6]:
class MapPlot: 
    def __init__(self, landmarks):
        
        self.plot = map_figure(x1=-60, x2=310, y1=-110, y2=60, h=600, w=900)
        plot_landmarks(self.plot, landmarks)
        
        self.robot_source = ColumnDataSource(data = { 'x' : [0], 'y' : [0], 'theta' : [0] })
        self.robot_range, self.robot_triangle, self.robot_cross = plot_initial_robot(self.plot, self.robot_source)
        
        stylize_plot(self.plot)
        self.show()
    
    def update(self, g):
        theta = -g.theta - 1.1 #FIX
        plot_robot(self.robot_triangle.data_source, g.x, g.y, theta)
        plot_robot(self.robot_cross.data_source, g.x, g.y, theta)
        plot_robot(self.robot_range.data_source, g.x, g.y, theta)
        push_notebook()
    
    def show(self):
        output_notebook()
        show(self.plot, notebook_handle=True)

In [7]:
class VicinityPlot: 
    def __init__(self, observations):
        self.plot = map_figure(x1=-70, x2=70, y1=-70, y2=70, h=250, w=250)
        self.plot_vehicle()

        observation_x, observation_y = extract_observations(observations)
        observation_source = ColumnDataSource({'x': observation_x, 'y': observation_y} )
        self.observation_squares = self.plot.square('x', 'y', legend="observations", source=observation_source,
            size=9, line_color=LANDMARK_COLOR, fill_alpha=0.0, line_width=1)
        
        stylize_plot(self.plot)
        self.show()  
    
    def plot_vehicle(self):
        robot_source = ColumnDataSource(data = { 'x' : [0], 'y' : [0], 'theta' : [-3.14 / 2] })
        self.plot.triangle('x', 'y', size=30, fill_color=ROBOT_FILL_COLOR,
            line_color=ROBOT_LINE_COLOR, fill_alpha = 1.0, line_width=1,
            angle = 'theta', legend = "ROBOT", source=robot_source)
        self.plot.cross('x', 'y', legend="ROBOT", source=robot_source,
            size=12, line_color=ROBOT_LINE_COLOR, angle = 'theta', fill_alpha=0.0, line_width=1)

    def update(self, observations):
        observation_x, observation_y = extract_observations(observations)
        self.observation_squares.data_source.data['x'] = observation_x
        self.observation_squares.data_source.data['y'] = observation_y
        push_notebook()
        
    def show(self):
        output_notebook()
        show(self.plot, notebook_handle=True)

In [ ]:
total_points = len(total_observations)
vicinity_plot = VicinityPlot(total_observations[0])

for i in range(1, total_points):    
    vicinity_plot.update(total_observations[i])
    sleep(0.01)


Loading BokehJS ...

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 10), ('y', 11)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 9), ('y', 10)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 10), ('y', 9)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 7), ('y', 8)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 6), ('y', 7)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 7), ('y', 6)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/sit

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 6), ('y', 5)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 7), ('y', 6)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 6), ('y', 7)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/sit

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 8), ('y', 9)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 9), ('y', 8)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 10), ('y', 9)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/si

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 9), ('y', 10)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 10), ('y', 9)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 9), ('y', 10)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/

/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 7), ('y', 8)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 6), ('y', 7)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 7), ('y', 6)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/mithi/miniconda3/envs/rusty-genes/lib/python3.6/sit

In [ ]:
total_points = len(groundtruths)
map_plot = MapPlot(landmarks)

for i in range(total_points):    
    map_plot.update(groundtruths[i])
    sleep(0.01)